## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv", index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kuhdasht,IR,33.5350,47.6061,46.27,62,0,2.37,clear sky
1,Kapaa,US,22.0752,-159.3190,80.58,74,40,1.01,scattered clouds
2,Kyra,RU,49.5776,111.9753,15.85,78,89,1.50,overcast clouds
3,Mar Del Plata,AR,-38.0023,-57.5575,64.09,70,0,5.99,clear sky
4,Albany,US,42.6001,-73.9662,28.74,62,1,1.99,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 40


In [4]:
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kuhdasht,IR,46.27,clear sky,33.5350,47.6061,
1,Kapaa,US,80.58,scattered clouds,22.0752,-159.3190,
3,Mar Del Plata,AR,64.09,clear sky,-38.0023,-57.5575,
5,Bogale,MM,83.98,scattered clouds,16.2833,95.4000,
6,Ushuaia,AR,42.46,broken clouds,-54.8000,-68.3000,
7,Riyadh,SA,59.14,clear sky,24.6877,46.7219,
8,Dingle,PH,82.85,overcast clouds,10.9995,122.6711,
9,Nacala,MZ,79.90,broken clouds,-14.5428,40.6728,
10,Codrington,AU,67.60,broken clouds,-38.2667,141.9667,
12,Aktau,KZ,45.18,broken clouds,43.6500,51.2000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kuhdasht,IR,46.27,clear sky,33.5350,47.6061,فروشگاه زنبیل
1,Kapaa,US,80.58,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Mar Del Plata,AR,64.09,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
5,Bogale,MM,83.98,scattered clouds,16.2833,95.4000,Aung Pin Lel Guest House
6,Ushuaia,AR,42.46,broken clouds,-54.8000,-68.3000,Albatros Hotel
...,...,...,...,...,...,...,...
706,Kumluca,TR,58.68,scattered clouds,36.3703,30.2869,TurkuazKöy
708,Desaguadero,PE,49.78,overcast clouds,-16.5656,-69.0417,av. panamericana 264 desaguadero hotel
709,Chibuto,MZ,68.94,overcast clouds,-24.6867,33.5306,Residência Masculina da Escola Superior de Neg...
712,Barra Do Garcas,BR,73.81,overcast clouds,-15.8900,-52.2567,Hotel Pousada Tropical


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



In [14]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))